# Adiabatic

## Factoring as a Constraint Satisfaction Problem
The [complexity class](https://en.wikipedia.org/wiki/Complexity_class) for classical integer factoring is believed to be between P and NP-hard.  Although research has yielded algorithms that perform faster than the intuitive trial division, including Fermat's algorithm, Pollard's two algorithms, and sieve algorithms, it's still an open question whether a classical algorithm exists that can factor in [polynomial time](https://en.wikipedia.org/wiki/Time_complexity). For quantum computing, Shor's algorithm runs in polynomial time (the D-Wave system does not run this algorithm).  

This notebook solves factoring on a D-Wave quantum computer by formulating it as a *[constraint satisfaction problem](https://docs.ocean.dwavesys.com/en/stable/concepts/csp.html)*. CSPs require that all a problem's variables be assigned values that result in the satisfying of all constraints. For factoring, the problem's constraints are that the two variables representing factors, $a$ and $b$, be assigned only natural numbers and that their multiplication be equal to the factored number, $P$. 

Among CSPs are hard problems well suited to solution on quantum computers. For example, the map-coloring problem is to color all regions of a map such that any two regions sharing a border have different colors (see a D-Wave system solve a four-color map-coloring problem here: [Ocean software examples](https://docs.ocean.dwavesys.com/en/stable/getting_started.html#examples)). The job-shop scheduling problem is to schedule multiple jobs done on several machines with constraints on the machines' execution of tasks. You can apply the solution technique shown here to many CSPs. 

In [ ]:
import dwavebinarycsp as dbc

In [3]:
P = 21  

bP = "{:06b}".format(P)    # "{:06b}" formats for 6-bit binary
print(bP)

010101


One option is to express the constraints of the problem with Boolean logic. This technique is very versatile: modern computing is built on Boolean gates, the scope of addressable problems is immense. The implementation below follows these steps:

1. Express $P=ab$ as a CSP with a binary multiplication circuit.
2. Convert to a BQM.
3. Program the quantum computer with the BQM's coefficients.

Some differences between the two formulations are noted below.

In [4]:
csp = dbc.factories.multiplication_circuit(3)

print(next(iter(csp.constraints)))

Constraint.from_configurations(frozenset({(1, 0, 0), (0, 0, 0), (1, 1, 1), (0, 1, 0)}), ('a0', 'b0', 'p0'), Vartype.BINARY, name='AND(a0, b0) = p0')


In [5]:
bqm = dbc.stitch(csp, min_classical_gap=.1)

print("BQM has {} variables: \n\t{}".format(len(bqm.variables), list(bqm.variables)))

BQM has 27 variables: 
	['a0', 'b0', 'p0', 'b1', 'and0,1', 'b2', 'and0,2', 'a1', 'and1,0', 'p1', 'carry1,0', 'and1,1', 'sum1,1', 'carry1,1', 'and1,2', 'a2', 'and2,0', 'p2', 'carry2,0', 'and2,1', 'sum2,1', 'carry2,1', 'and2,2', 'p3', 'carry3,0', 'p4', 'p5']


In [13]:
p_vars = ['p0', 'p1', 'p2', 'p3', 'p4', 'p5']

# Convert P from decimal to binary
fixed_variables = dict(zip(reversed(p_vars), "{:06b}".format(P)))
fixed_variables = {var: int(x) for(var, x) in fixed_variables.items()}

# Fix product variables
for var, value in fixed_variables.items():
    bqm.fix_variable(var, value)
    
print("BQM has {} non-fixed variables: \n\t{}".format(len(bqm.variables), list(bqm.variables)))

BQM has 21 non-fixed variables: 
	['a0', 'b0', 'b1', 'and0,1', 'b2', 'and0,2', 'a1', 'and1,0', 'carry1,0', 'and1,1', 'sum1,1', 'carry1,1', 'and1,2', 'a2', 'and2,0', 'carry2,0', 'and2,1', 'sum2,1', 'carry2,1', 'and2,2', 'carry3,0']


In [14]:
from dwave.system import DWaveSampler

# Use a D-Wave system as the sampler
sampler = DWaveSampler() 

print("QPU {} was selected.".format(sampler.solver.name))

QPU Advantage_system1.1 was selected.


In [15]:
from dwave.system import EmbeddingComposite

embedding_sampler = EmbeddingComposite(sampler)

In [16]:
sampleset = embedding_sampler.sample(bqm, num_reads=100, label='Notebook - Factoring')

print("Best solution found: \n",sampleset.first.sample)

Best solution found: 
 {'a0': 1, 'a1': 1, 'a2': 1, 'and0,1': 1, 'and0,2': 0, 'and1,0': 1, 'and1,1': 1, 'and1,2': 0, 'and2,0': 1, 'and2,1': 1, 'and2,2': 0, 'b0': 1, 'b1': 1, 'b2': 0, 'carry1,0': 1, 'carry1,1': 0, 'carry2,0': 1, 'carry2,1': 0, 'carry3,0': 1, 'sum1,1': 1, 'sum2,1': 1}


In [17]:
def to_base_ten(sample):
    a = b = 0
    
    # we know that multiplication_circuit() has created these variables
    a_vars = ['a0', 'a1', 'a2']
    b_vars = ['b0', 'b1', 'b2']
    
    for lbl in reversed(a_vars):
        a = (a << 1) | sample[lbl]
    for lbl in reversed(b_vars):
        b = (b << 1) | sample[lbl] 
        
    return a,b
 
a, b = to_base_ten(sampleset.first.sample)

print("Given integer P={}, found factors a={} and b={}".format(P, a, b))

Given integer P=21, found factors a=7 and b=3
